In [2]:
# connect to drive to get the trained model
from google.colab import drive

drive.mount('/content/gdrive')

Mounted at /content/gdrive


In [ ]:
#unzip the model 
#!gzip -d "/content/gdrive/MyDrive/contexto/GoogleNews-vectors-negative300.bin.gz"

In [3]:
#copy the model to my colab
!cp "/content/gdrive/MyDrive/contexto/GoogleNews-vectors-negative300.bin" "/content/"

In [2]:
pip install gensim

     ---------------------------------------- 24.0/24.0 MB 3.0 MB/s eta 0:00:00
     ---------------------------------------- 56.8/56.8 kB 1.5 MB/s eta 0:00:00
Note: you may need to restart the kernel to use updated packages.



[notice] A new release of pip available: 22.3.1 -> 23.0.1
[notice] To update, run: C:\Users\ое\AppData\Local\Microsoft\WindowsApps\PythonSoftwareFoundation.Python.3.10_qbz5n2kfra8p0\python.exe -m pip install --upgrade pip


In [3]:
#create the model
import numpy as np
import gensim
modelPath="D:\\projectContexto\\contextoApp\\GoogleNews-vectors-negative300.bin"
wordsPath= "D:\\projectContexto\\contextoApp\\nouns.txt"
model= gensim.models.keyedvectors.Word2VecKeyedVectors.load_word2vec_format(modelPath, binary=True)


In [4]:
#get the most common words in english

data = [line.strip() for line in open(wordsPath, 'r')]
data[100]

'gas'

In [5]:
print(model.similarity(data[1], data[100]))

0.31306836


In [6]:
vocabs= list(model.key_to_index.keys())
#vocabs

In [7]:
class environment():
  def __init__(self, available_words, model):
    self.model= model #gensim.models.keyedvectors.Word2VecKeyedVectors.load_word2vec_format('/content/GoogleNews-vectors-negative300.bin', binary=True)
    self.observation=  dict()
    self.reward=0
    self.done= False
    self.action= " "
    self.action_space= available_words
    self.target_word=np.random.choice(self.action_space)
    self.attempts= 0
  
  def reset(self):
    self.target_word=np.random.choice(self.action_space)
    self.observation=  dict()
    self.reward=0
    self.done= False
    self.action= " "
    self.attempts= 0
    return self.observation, self.reward, self.done


  def act(self, action):
    if not self.done:
      if action not in self.action_space:
        print("word not in avilable words")
        self.reward= -1000
        self.observation[action]= self.reward 
        self.attempts+= 1
        return self.observation, self.reward, self.done
      else:
        similarity= self.model.similarity(action, self.target_word)
        self.reward= similarity*100
        self.observation[action]= similarity
        if (similarity>= 0.999):
          self.done= True
        return self.observation, self.reward, self.done
    else:
      self.observation, self.reward, self.done= self.reset()
      return self.observation, self.reward, self.done

  def as_len(self):
      return len(self.action_space)

In [13]:
#try the environment 
contexto= environment(data, model)

print(len(contexto.observation.items()))

0


In [21]:
w='building'
distances=dict()
for w in data:
  if w in vocabs:
    distances[w]= model.distance(w, contexto.target_word)

#distances

In [22]:
min(distances.values())
print(list(distances.keys())[list(distances.values()).index(min(distances.values()))])  # Prints george

awareness


In [14]:
obs, reward, done =contexto.act("awareness")
print(obs, reward, done)

{'awareness': 0.1511945} 15.11944979429245 False


In [22]:
for item in obs.items():
    print(item[0])

awareness


In [24]:
guessed= dict()

In [30]:

guess= "ge"

guessed[guess]= 1

guessed


{'table': 1, 'book': 1, 'ge': 1}

In [27]:
import pandas as pd
df=pd.DataFrame.from_dict(guessed, orient='index')

In [28]:
df

,0
table,1
book,1


creating the agent

In [61]:
import matplotlib
import matplotlib.pyplot as plt
from collections import namedtuple, deque
from itertools import count
import math


import torch
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F

# set up matplotlib
is_ipython = 'inline' in matplotlib.get_backend()
if is_ipython:
    from IPython import display

plt.ion()

# if gpu is to be used
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

Q-learning attempt

In [60]:
#a simple Q-learning model should be able to solve this game
q_table= np.zeros((contexto.as_len(),contexto.as_len()))
#fill all table values with -100
q_table= np.full((contexto.as_len(),contexto.as_len()))
q_table

array([[-100, -100, -100, ..., -100, -100, -100],
       [-100, -100, -100, ..., -100, -100, -100],
       [-100, -100, -100, ..., -100, -100, -100],
       ...,
       [-100, -100, -100, ..., -100, -100, -100],
       [-100, -100, -100, ..., -100, -100, -100],
       [-100, -100, -100, ..., -100, -100, -100]])

The UI

In [24]:
pip install kivy


  Consider adding this directory to PATH or, if you prefer to suppress this warning, use --no-warn-script-location.

[notice] A new release of pip available: 22.3.1 -> 23.0.1
[notice] To update, run: C:\Users\ое\AppData\Local\Microsoft\WindowsApps\PythonSoftwareFoundation.Python.3.10_qbz5n2kfra8p0\python.exe -m pip install --upgrade pip



     ---------------------------------------- 4.0/4.0 MB 7.9 MB/s eta 0:00:00
     ---------------------------------------- 4.8/4.8 MB 7.5 MB/s eta 0:00:00
     -------------------------------------- 570.5/570.5 kB 4.5 MB/s eta 0:00:00
     ---------------------------------------- 3.1/3.1 MB 7.2 MB/s eta 0:00:00
     -------------------------------------- 123.6/123.6 kB 3.7 MB/s eta 0:00:00


In [54]:
import pandas as pd

guessesPath= "D:\\projectContexto\\contextoApp\\guesses.txt"


pdguessed= pd.read_csv(guessesPath, sep=",", header=0)


In [76]:
#pdguessed.iloc[:,["Words" ,"Similarities"]]

def highlight_survived(s):
    #print(type(s))
    print("#########")
    #print( ['background-color: green']*len(s) if s.Words =='word' else ['background-color: red']*len(s))

#newl =pd.DataFrame(pdguessed.style.apply(highlight_survived, axis=1))
pdguessed= pd.read_csv(guessesPath, sep=",", header=0)
 
 


def color(x):
    cold= 0
    worm= 0.4
    hot= 0.7
    print(x[1])
    if  x[1]>= cold and x[1]<= worm :
        return ['background-color : #e81e80']*len(x)
    elif x[1]> worm and x[1]<= hot:
        return ['background-color : #Ea7051']*len(x)
    elif x[1]> hot:
        return ['background-color : #73f181']*len(x)


sortedGuesses=pdguessed.sort_values(by=[pdguessed.keys()[1]], ascending=False) 
#sortedGuesses.style.apply(color, axis=1)

pdguessed.keys()[1]

' Similarities'

In [ ]:
"""

for i in range(len(pdguessed)):
    print(pdguessed.iloc[[i],[0]])"""